test file for nlp packages that predict race to see acuuracy,the following are the packages using 

1. ethnicolr
The model uses US census data, the Florida voting registration data, and the Wikipedia data collected by Skiena and colleagues, to predict race and ethnicity based on first and last name or just the last name. 
The model provides 06 functions. 

Source: https://ethnicolr.readthedocs.io/ethnicolr.html#install  

2. rethnicity
The model is a Bidirectional Long Short-Term Memory (Bi-LSTM), a recurrent neural network architecture commonly used for natural language processing, was chosen as the model for our study. The Florida Voter Registration was used as the training and testing data. 

Source: https://cran.r-project.org/web/packages/rethnicity/rethnicity.pdf 


The Prediction Process: 

owner ----> nlp ----> W: 70% NW: 30%    
geoid:059.... ----> W: 80% NW: 20%
            .
            .
            .
            .
geoid:059.... ----> W: 60% NW: 40%
            and use
        min & max scale? 
          ex.  
            1 -----> 40

In [1]:
import pandas as pd


#data fram we will use to test the nlp models
mergent_true_df = pd.read_csv('../data/mergent_intellect_executives/mergent_executive_flagged.csv.xz')
mergent_true_df.columns #ask about search column, what does this mean 

#data frame with just the first and the last names
namesdf = pd.DataFrame(mergent_true_df, columns=['lastname', 'firstname'])
namesdf
#the orignal data frame contains 1067 names

,lastname,firstname
0,Abod,Craig
1,Soliday,Brian
2,Abod,Craig
3,Maigue,Richard
4,Mills,Mark
...,...,...
18151,French,Pamela
18152,Kelly,Leesa
18153,Donner,Tim
18154,Hakeem,Dirar


# Ethnicolor:

In [2]:
import ethnicolr
from ethnicolr import census_ln, pred_census_ln

#this is so i don't have to write it out
f = 'firstname'
l = 'lastname'

fname = pd.DataFrame(mergent_true_df, columns=[f]) #data frame with just first names
lname = pd.DataFrame(mergent_true_df, columns=[l]) #data frame with just last names
namesdf #datafram with both first and last names

2023-07-10 14:13:35.885577: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-10 14:13:35.888286: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 14:13:35.938454: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 14:13:35.939388: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 14:13:37.378167: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

,lastname,firstname
0,Abod,Craig
1,Soliday,Brian
2,Abod,Craig
3,Maigue,Richard
4,Mills,Mark
...,...,...
18151,French,Pamela
18152,Kelly,Leesa
18153,Donner,Tim
18154,Hakeem,Dirar


## censu_ln w/first

In [3]:
w = census_ln(fname, f, 2010)

#produces a sig. amount of null values
#does not predict the race automatically

## censu_ln w/last

In [4]:
x = census_ln(fname, f, 2010)

#output is mostly null

## pred_census_ln w firstnames

In [5]:
fdf = pred_census_ln(fname, f, 2010)

#as you can see this model outputs only 615 predictions

2023-07-10 14:13:42.767776: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


### sort save as ethnicolor_censu_fn_pred.csv

In [6]:
#sort to binary classification w first names
#using the pred_census_fn(fname, l, 2010) data frame w/ prediction, i am transforming the data into something useful for the proj. 
trandf = pd.DataFrame(fdf, columns=[f, 'white'])
notw = list(1-trandf['white'])
trandf['not_white'] = notw

race = list(list(fdf['race']))

newlist = []
for r in race:
    newlist.append(str(r))
newlist

correct = []
for i in newlist:
    if i != 'white':
       #print(i,True)
       correct.append(1) 
    if i == 'white':
       #print(i, False)
       correct.append(0) 
correct

trandf['race'] = correct
trandf

#I added a prop not_white column and made it so that the race column used boolean values:
#1 if minority 
#0 if white

#i can do this for the firstname input of the function upon request as well

#save as ethnicolor_censu_fn_predictions.csv
#615 results, not many null values

,firstname,white,not_white,race
0,Craig,0.869542,0.130458,0
1,Brian,0.674150,0.325850,0
2,Richard,0.745203,0.254797,0
3,Mark,0.669779,0.330221,0
4,John,0.500118,0.499882,0
...,...,...,...,...
4526,Kambiz,0.556906,0.443094,0
4527,Wesam,0.733945,0.266055,0
4528,Jal,0.443175,0.556825,0
4529,Teel,0.852241,0.147759,0


## pred_census_ln w lastnames

In [7]:
ldf = pred_census_ln(lname, l, 2010)

#actually predicts race in an output
#first names work too but the output is diff and produces smaller row number (predictions) - 615 to be exact

### sort save as ethnicolor_censu_ln_pred.csv

In [8]:

#using the pred_census_ln(lname, l, 2010) data frame w/ prediction, i am transforming the data into something useful for the proj. 
transdf = pd.DataFrame(ldf, columns=[l, 'white'])
notw = list(1-transdf['white'])
transdf['not_white'] = notw

race = list(list(ldf['race']))

newlist = []
for r in race:
    newlist.append(str(r))
newlist

correct = []
for i in newlist:
    if i != 'white':
       #print(i,True)
       correct.append(1) 
    if i == 'white':
       #print(i, False)
       correct.append(0) 
correct

transdf['race'] = correct
transdf


#i can do this for the firstname input of the function upon request as well
#save as ethnicolor_censu_ln_predictions.csv
#916 results no visible null values

,lastname,white,not_white,race
0,Abod,0.756722,0.243278,0
1,Soliday,0.656425,0.343575,0
2,Maigue,0.808320,0.191680,0
3,Mills,0.793126,0.206874,0
4,Lee,0.360847,0.639153,1
...,...,...,...,...
10192,Fondaco,0.776765,0.223235,0
10193,Adeler,0.774511,0.225489,0
10194,Faraone,0.804526,0.195474,0
10195,Cryan,0.887995,0.112005,0


In [16]:
d = pd.DataFrame(namesdf, columns=[l, f])
m = trandf['not_white'] * transdf['not_white']
m
no = d.dropna
d['not_white'] = m 
d

,lastname,firstname,not_white
0,Abod,Craig,0.031738
1,Soliday,Brian,0.111954
2,Abod,Craig,0.048840
3,Maigue,Richard,0.068314
4,Mills,Mark,0.319501
...,...,...,...
18151,French,Pamela,NaN
18152,Kelly,Leesa,NaN
18153,Donner,Tim,NaN
18154,Hakeem,Dirar,NaN


## pred_wiki_ln w/ firstnames

In [17]:
from ethnicolr import pred_wiki_ln, pred_wiki_name
wikifirstdf = pred_wiki_ln(fname, f, num_iter=100, conf_int=.90)
#i like how this function has a confidence interval input 
wikifirstdf

white = ['GreaterEuropean,WestEuropean,Italian_mean', 'GreaterEuropean,WestEuropean,Nordic_mean', 
          'GreaterEuropean,WestEuropean,Germanic_mean', 'GreaterEuropean,WestEuropean,French_mean',
          'GreaterEuropean,Jewish_mean', 'GreaterEuropean,British_mean','GreaterEuropean,EastEuropean_mean']
whitelist = ['GreaterEuropean,WestEuropean,Italian', 'GreaterEuropean,WestEuropean,Nordic',
         'GreaterEuropean,WestEuropean,Germanic', 'GreaterEuropean,WestEuropean,French',
          'GreaterEuropean,Jewish', 'GreaterEuropean,British', 'GreaterEuropean,EastEuropean']

#this does not produce many predictions with the first name --> 615


['Asian,GreaterEastAsian,EastAsian', 'Asian,GreaterEastAsian,Japanese', 'Asian,IndianSubContinent', 'GreaterAfrican,Africans', 'GreaterAfrican,Muslim', 'GreaterEuropean,British', 'GreaterEuropean,EastEuropean', 'GreaterEuropean,Jewish', 'GreaterEuropean,WestEuropean,French', 'GreaterEuropean,WestEuropean,Germanic', 'GreaterEuropean,WestEuropean,Hispanic', 'GreaterEuropean,WestEuropean,Italian', 'GreaterEuropean,WestEuropean,Nordic']


### sort save as ethnicolor_wiki_fn_pred.csv

In [18]:
#sort into binary classification

testfndf = pd.DataFrame(wikifirstdf, columns=[f])
testfndf['white'] = wikifirstdf[white].sum(axis=1)
testfndf['not_white'] = 1- testfndf['white']
testfndf['race'] = wikifirstdf['race']


racelistfn = list(testfndf['race'])

race = []

for r in racelistfn:
    if r not in whitelist:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)

testfndf['race'] = race
testfndf
#save as ethnicolor_wiki_fn_predictions.csv
#null values:
#defalut is 1.00 not_white and for thre race column thats automatcially gunna be 1 

,firstname,white,not_white,race
0,Craig,0.269543,0.730457,1
1,Brian,0.362300,0.637700,1
2,Richard,0.476099,0.523901,0
3,Mark,0.448163,0.551837,1
4,John,0.287773,0.712227,1
...,...,...,...,...
4517,Kambiz,0.000000,1.000000,1
4518,Wesam,0.000000,1.000000,1
4519,Jal,0.000000,1.000000,1
4520,Teel,0.000000,1.000000,1


## pred_wiki_ln w/lastnames

In [19]:
wikilastdf = pred_wiki_ln(lname, l, num_iter=100, conf_int=.90)

#this produces more predictions than the first name -->917

['Asian,GreaterEastAsian,EastAsian', 'Asian,GreaterEastAsian,Japanese', 'Asian,IndianSubContinent', 'GreaterAfrican,Africans', 'GreaterAfrican,Muslim', 'GreaterEuropean,British', 'GreaterEuropean,EastEuropean', 'GreaterEuropean,Jewish', 'GreaterEuropean,WestEuropean,French', 'GreaterEuropean,WestEuropean,Germanic', 'GreaterEuropean,WestEuropean,Hispanic', 'GreaterEuropean,WestEuropean,Italian', 'GreaterEuropean,WestEuropean,Nordic']


### sort save as ethnicolor_wiki_ln_pred.csv

In [20]:
#sort into binary classification



testlndf = pd.DataFrame(wikilastdf, columns=[l])
testlndf['white'] = wikilastdf[white].sum(axis=1)
testlndf['not_white'] = 1- testlndf['white']
testlndf['race'] = wikilastdf['race']


racelistln = list(testlndf['race'])

race = []

for r in racelistln:
    if r not in whitelist:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)

testlndf['race'] = race
testlndf
#save as ethnicolor_wiki_last_predictions.csv
#null values:
#defalut is 1.00 not_white and for thre race column thats automatcially gunna be 1 

,lastname,white,not_white,race
0,Abod,0.818203,0.181797,0
1,Soliday,0.787148,0.212852,0
2,Maigue,0.976047,0.023953,0
3,Mills,0.114507,0.885493,1
4,Lee,0.089706,0.910294,1
...,...,...,...,...
10164,Fondaco,0.000000,1.000000,1
10165,Adeler,0.000000,1.000000,1
10166,Faraone,0.000000,1.000000,1
10167,Cryan,0.000000,1.000000,1


## pred_wiki_name (this takes both the first and last names)

In [21]:
from ethnicolr import pred_wiki_ln, pred_wiki_name
wikifulldf = pred_wiki_name(namesdf,l, f, conf_int=0.9)


#produces some null values but it does list most items from the original dataframe, most likely because it uses both names  



['Asian,GreaterEastAsian,EastAsian', 'Asian,GreaterEastAsian,Japanese', 'Asian,IndianSubContinent', 'GreaterAfrican,Africans', 'GreaterAfrican,Muslim', 'GreaterEuropean,British', 'GreaterEuropean,EastEuropean', 'GreaterEuropean,Jewish', 'GreaterEuropean,WestEuropean,French', 'GreaterEuropean,WestEuropean,Germanic', 'GreaterEuropean,WestEuropean,Hispanic', 'GreaterEuropean,WestEuropean,Italian', 'GreaterEuropean,WestEuropean,Nordic']


### sort save as ethnicolor_wiki_full_pred.csv

In [22]:
#sort into binary classification
#output is ethnicity, here is the list, each have mean, upper, lower bound, and standard error, we will be using the mean 
#percentages
# ethnicitylist = ['Asian,GreaterEastAsian,EastAsian', 'Asian,GreaterEastAsian,Japanese',
# 'Asian,IndianSubContinent', 'GreaterAfrican,Africans', 'GreaterAfrican,Muslim',
#  'GreaterEuropean,British', 'GreaterEuropean,EastEuropean',
#  'GreaterEuropean,Jewish', 'GreaterEuropean,WestEuropean,French', 
#  'GreaterEuropean,WestEuropean,Germanic', 'GreaterEuropean,WestEuropean,Hispanic', 
#  'GreaterEuropean,WestEuropean,Italian', 'GreaterEuropean,WestEuropean,Nordic']


a = pd.DataFrame(wikifulldf, columns=[f, l])


a['white'] = wikifulldf[white].sum(axis=1)
a['not_white'] = 1- a['white']
a['race'] = wikifulldf['race']



racelist = list(a['race'])




race = []

for r in racelist:
    if r in whitelist:
        #print(True)
        race.append(0)
    else:
        #print(False)  
        race.append(1)
 

a['race'] = race
a

#save as ethnicolor_wiki_full_predictions.csv
#null values:
#defalut is 1.00 not_white and for thre race column thats automatcially gunna be 1 

,firstname,lastname,white,not_white,race
0,Craig,Abod,0.965115,0.034885,0
1,Brian,Soliday,0.927882,0.072118,0
2,Richard,Maigue,0.979862,0.020138,0
3,Mark,Mills,0.857443,0.142557,0
4,John,Lee,0.187918,0.812082,1
...,...,...,...,...,...
16305,Michael,Arthur,0.000000,1.000000,1
16306,Pamela,French,0.000000,1.000000,1
16307,Leesa,Kelly,0.000000,1.000000,1
16308,Tim,Donner,0.000000,1.000000,1


## pred_fl_reg_ln(df, namecol, num_iter=100, conf_int=1.0) w/lastname

In [23]:
from ethnicolr import pred_fl_reg_ln, pred_fl_reg_name, pred_fl_reg_ln_five_cat, pred_fl_reg_name_five_cat
odf = pred_fl_reg_ln(ldf, l, num_iter=100, conf_int=.90)

#Output: Appends the following columns to the pandas DataFrame or CSV: 
#race (white, black, asian, or hispanic), asian (percentage chance Asian), hispanic, nh_black, nh_white.
#For each race it will provide the mean, standard error, lower & upper bound of confidence interval

['asian', 'hispanic', 'nh_black', 'nh_white']


### sort save as ethnicolor_fl_ln_pred.csv

In [24]:
tstdf = pd.DataFrame( columns=[l, 'white'])
tstdf
ls = list(lname[l])
tstdf[l] = ls
tstdf['white'] = odf['white']
tstdf['not_white'] = 1 - tstdf['white']
tstdf['race'] = odf['race_x']

racelist = list(tstdf['race'])

race = []

for r in racelist:
    if r in 'white':
        #print(True)
        race.append(0)
    else:
        #print(False)  
        race.append(1)
 

tstdf['race'] = race
tstdf

#917 results, don't see null values 

,lastname,white,not_white,race
0,Abod,0.756722,0.243278,0
1,Soliday,0.656425,0.343575,0
2,Maigue,0.808320,0.191680,0
3,Mills,0.793126,0.206874,0
4,Lee,0.360847,0.639153,1
...,...,...,...,...
10164,Fondaco,0.776765,0.223235,0
10165,Adeler,0.774511,0.225489,0
10166,Faraone,0.804526,0.195474,0
10167,Cryan,0.887995,0.112005,0


##  pred_fl_reg_name(df, 'last', 'first', conf_int=0.9) w fullname

In [25]:
odf = pred_fl_reg_name(namesdf, l, f, conf_int=0.9)



['asian', 'hispanic', 'nh_black', 'nh_white']


### sort save as ethnicolor_fl_full_pred.csv

In [26]:
b = pd.DataFrame(odf, columns=[f, l])
b['white'] = odf['nh_white_mean']
b['not_white'] = 1 - b['white']
b['race'] = odf['race']

racelist = list(b['race'])
race = []

        

for r in racelist:
    if 'white' in str(r):
        #print(True)
        race.append(0)

    else:
        #print(False)  
        race.append(1)
 

b['race'] = race
b

,firstname,lastname,white,not_white,race
0,Craig,Abod,0.896616,0.103384,0
1,Brian,Soliday,0.739485,0.260515,0
2,Richard,Maigue,0.891501,0.108499,0
3,Mark,Mills,0.763613,0.236387,0
4,John,Lee,0.236917,0.763083,1
...,...,...,...,...,...
16305,Michael,Arthur,NaN,NaN,1
16306,Pamela,French,NaN,NaN,1
16307,Leesa,Kelly,NaN,NaN,1
16308,Tim,Donner,NaN,NaN,1


In [27]:
#3 more functions then next package finally 

## pred_fl_reg_ln_five_cat takes a last name input but ends up using the full name 

In [28]:
cdf = pred_fl_reg_ln_five_cat(namesdf, l, num_iter=100, conf_int=0.9)


['asian', 'hispanic', 'nh_black', 'nh_white', 'other']


### sort and save as ethnicolor_fl_full2_csv

In [29]:
c = pd.DataFrame(cdf, columns=[f, l])
c['white'] = cdf['nh_white_mean']
c['not_white'] = 1 - c['white']
c['race'] = cdf['race']

racelist = list(c['race'])
race = []

        

for r in racelist:
    if 'white' in str(r):
        #print(True)
        race.append(0)

    else:
        #print(False)  
        race.append(1)
 

c['race'] = race
c

,firstname,lastname,white,not_white,race
0,Craig,Abod,0.177798,0.822202,1
1,Brian,Soliday,0.093042,0.906958,1
2,Richard,Maigue,0.227647,0.772353,1
3,Mark,Mills,0.081415,0.918585,1
4,John,Lee,0.022998,0.977002,1
...,...,...,...,...,...
10185,Sal,Fondaco,NaN,NaN,1
10186,George,Adeler,NaN,NaN,1
10187,Debra,Faraone,NaN,NaN,1
10188,Eric,Cryan,NaN,NaN,1


## pred_fl_reg_name_five_cat(df, namecol, num_iter=100, conf_int=1.0) w/full name input

In [30]:
rdf = pred_fl_reg_name_five_cat(namesdf, l, f, num_iter=100, conf_int=0.9)


['asian', 'hispanic', 'nh_black', 'nh_white', 'other']


In [31]:
c = pd.DataFrame(rdf, columns=[f, l])
c['white'] = rdf['nh_white_mean']
c['not_white'] = 1 - c['white']
c['race'] = rdf['race']

racelist = list(c['race'])
race = []

        

for r in racelist:
    if 'white' in str(r):
        #print(True)
        race.append(0)

    else:
        #print(False)  
        race.append(1)
 

c['race'] = race
c

,firstname,lastname,white,not_white,race
0,Craig,Abod,0.547530,0.452470,0
1,Brian,Soliday,0.327795,0.672205,0
2,Richard,Maigue,0.490032,0.509968,0
3,Mark,Mills,0.210264,0.789736,1
4,John,Lee,0.020661,0.979339,1
...,...,...,...,...,...
10185,Sal,Fondaco,NaN,NaN,1
10186,George,Adeler,NaN,NaN,1
10187,Debra,Faraone,NaN,NaN,1
10188,Eric,Cryan,NaN,NaN,1


## pred_nc_reg_name(df, namecol, num_iter=100, conf_int=1.0) w/ full names 
### but it says nc here...are they using north carolina voter registration data? I looked all over the website as well. I tried to load a page that was linked to the "full name NC model" to see what it was about and it returned an error

In [32]:
from ethnicolr import pred_nc_reg_name
z = pred_nc_reg_name(namesdf, l, f, num_iter=100, conf_int=.90)
z

2023-07-10 14:31:41.990371: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-10 14:31:41.992644: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-10 14:31:41.993840: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

['HL+A', 'HL+B', 'HL+I', 'HL+M', 'HL+O', 'HL+W', 'NL+A', 'NL+B', 'NL+I', 'NL+M', 'NL+O', 'NL+W']


,lastname,firstname,HL+A_mean,HL+A_std,HL+A_lb,HL+A_ub,HL+B_mean,HL+B_std,HL+B_lb,HL+B_ub,...,NL+M_ub,NL+O_mean,NL+O_std,NL+O_lb,NL+O_ub,NL+W_mean,NL+W_std,NL+W_lb,NL+W_ub,race
0,Abod,Craig,2.833178e-11,0.0,2.833178e-11,2.833178e-11,1.299032e-08,0.0,1.299032e-08,1.299032e-08,...,3.097475e-06,0.275574,0.0,0.275574,0.275574,0.313212,0.0,0.313212,0.313212,NL+W
1,Soliday,Brian,1.344512e-11,0.0,1.344512e-11,1.344512e-11,4.788999e-11,0.0,4.788999e-11,4.788999e-11,...,1.038537e-10,0.008347,0.0,0.008347,0.008347,0.000579,0.0,0.000579,0.000579,HL+M
2,Maigue,Richard,1.238406e-10,0.0,1.238406e-10,1.238406e-10,2.196076e-07,0.0,2.196076e-07,2.196076e-07,...,7.705532e-04,0.036048,0.0,0.036048,0.036048,0.108246,0.0,0.108246,0.108246,NL+I
3,Mills,Mark,8.189762e-08,0.0,8.189762e-08,8.189762e-08,1.556497e-05,0.0,1.556497e-05,1.556497e-05,...,9.926235e-04,0.062795,0.0,0.062795,0.062795,0.018194,0.0,0.018194,0.018194,NL+A
4,Lee,John,7.917655e-13,0.0,7.917655e-13,7.917655e-13,7.453077e-04,0.0,7.453077e-04,7.453077e-04,...,3.132217e-04,0.660733,0.0,0.660733,0.660733,0.009487,0.0,0.009487,0.009487,NL+O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10185,Fondaco,Sal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10186,Adeler,George,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10187,Faraone,Debra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10188,Cryan,Eric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# / ***************************************************************************
# Race codes
# race               description
# *******************************************************************************
# A                  ASIAN
# B                  BLACK or AFRICAN AMERICAN
# I                  AMERICAN INDIAN or ALASKA NATIVE
# M                  TWO or MORE RACES
# O                  OTHER
# P                  NATIVE HAWAIIAN or PACIFIC ISLANDER
# U                  UNDESIGNATED
# W                  WHITE
# *************************************************************************** /

# / ***************************************************************************
# Ethnic codes
# ethnicity          description
# *******************************************************************************
# HL                 HISPANIC or LATINO
# NL                 NOT HISPANIC or NOT LATINO
# UN                 UNDESIGNATED
# *************************************************************************** /

### sort and save as ethnicolor_nc_full_pred.csv

In [34]:
ddf = pd.DataFrame(a, columns=[f, l])
ddf['white'] =  z['NL+W'] + z['HL+W']
ddf['not_white'] = 1 - ddf['white']
ddf['race'] = z['race']

racelist = list(ddf['race'])
race = []

        

for r in racelist:
    if 'W' in str(r):
        #print(True)
        race.append(0)

    else:
        #print(False)  
        race.append(1)
 

ddf['race'] = race
ddf

#this doesn't look correct at all. Majority of these names should not be predicted as a minority

KeyError: 'NL+W'

# Done with testing

In [ ]:
mergent_true_df['prob_nw_eth_wiki'] = a['not_white']
mergent_true_df['prob_nw_eth_fl_reg_name'] = b['not_white']
mergent_true_df['prob_nw_eth_fl_five_cat'] = c['not_white']
mergent_true_df['prob_nw_eth_census'] = d['not_white']
mergent_true_df
# nonull = mergent_true_df. dropna() 
# nonull